In [1]:
# -*- coding: utf-8 -*-
"""
Created on Fri Sep 27 02:04:36 2024

@author: Yunus
"""



'\nCreated on Fri Sep 27 02:04:36 2024\n\n@author: Yunus\n'

In [2]:
# pip install ucimlrepo

In [3]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, roc_auc_score

# Load a real dataset (example: UCI Spambase dataset)
# Dataset available here: https://archive.ics.uci.edu/ml/datasets/spambase
# df = pd.read_csv('spambase.data', header=None)

from ucimlrepo import fetch_ucirepo 
  
# fetch dataset 
spambase = fetch_ucirepo(id=94) 
  
# data (as pandas dataframes) 
X = spambase.data.features 
y = spambase.data.targets 
  
# metadata 
print(spambase.metadata) 
  
# variable information 
print(spambase.variables) 

{'uci_id': 94, 'name': 'Spambase', 'repository_url': 'https://archive.ics.uci.edu/dataset/94/spambase', 'data_url': 'https://archive.ics.uci.edu/static/public/94/data.csv', 'abstract': 'Classifying Email as Spam or Non-Spam', 'area': 'Computer Science', 'tasks': ['Classification'], 'characteristics': ['Multivariate'], 'num_instances': 4601, 'num_features': 57, 'feature_types': ['Integer', 'Real'], 'demographics': [], 'target_col': ['Class'], 'index_col': None, 'has_missing_values': 'no', 'missing_values_symbol': None, 'year_of_dataset_creation': 1999, 'last_updated': 'Mon Aug 28 2023', 'dataset_doi': '10.24432/C53G6X', 'creators': ['Mark Hopkins', 'Erik Reeber', 'George Forman', 'Jaap Suermondt'], 'intro_paper': None, 'additional_info': {'summary': 'The "spam" concept is diverse: advertisements for products/web sites, make money fast schemes, chain letters, pornography...\n\nThe classification task for this dataset is to determine whether a given email is spam or not.\n\t\nOur collecti

In [4]:
# Split into features and labels
# X = df.iloc[:, :-1].values  # Features
# y = df.iloc[:, -1].values   # Labels (spam or not spam)

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Correcting y_train and y_test shapes
y_train = y_train.values.ravel()  # Flatten the target variable to 1D array
y_test = y_test.values.ravel()

# Train multiple classifiers (to simulate predictions from multiple annotators)
clf1 = RandomForestClassifier(n_estimators=100, random_state=42)
clf2 = SVC(probability=True, random_state=42)
clf3 = LogisticRegression(max_iter=1000, random_state=42)

# Fit the classifiers on training data
clf1.fit(X_train, y_train)
clf2.fit(X_train, y_train)
clf3.fit(X_train, y_train)

# Get probabilistic predictions (simulating annotator probabilities)
pred1 = clf1.predict_proba(X_test)[:, 1]  # Probability of spam (class 1)
pred2 = clf2.predict_proba(X_test)[:, 1]
pred3 = clf3.predict_proba(X_test)[:, 1]

# Create a DataFrame of predictions from different classifiers
df_preds = pd.DataFrame({'Model_1': pred1, 'Model_2': pred2, 'Model_3': pred3})
print(df_preds.head())



# Now apply logistic regression to combine these predictions (Platt-Burges approach)
aggregator = LogisticRegression(max_iter=2000, random_state=42)
aggregator.fit(df_preds, y_test)

# Get the final aggregated prediction
final_preds = aggregator.predict_proba(df_preds)[:, 1]  # Final probabilities of spam (class 1)
final_classification = np.where(final_preds >= 0.5, 1, 0)

# Evaluate the results
accuracy = accuracy_score(y_test, final_classification)
auc_score = roc_auc_score(y_test, final_preds)

print(f"Platt-Burges Aggregation Accuracy: {accuracy:.4f}")
print(f"Platt-Burges Aggregation AUC Score: {auc_score:.4f}")


C:\Users\yunus\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:469: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


   Model_1   Model_2   Model_3
0     0.00  0.212372  0.177094
1     0.26  0.214712  0.159391
2     0.08  0.235542  0.026408
3     0.65  0.457330  0.966633
4     0.00  0.251327  0.073418
Platt-Burges Aggregation Accuracy: 0.9558
Platt-Burges Aggregation AUC Score: 0.9896


In [5]:
from scipy.stats import mode

# Majority Voting
majority_vote = np.where((pred1 >= 0.5).astype(int) + (pred2 >= 0.5).astype(int) + (pred3 >= 0.5).astype(int) > 1, 1, 0)
accuracy_mv = accuracy_score(y_test, majority_vote)
print(f"Majority Voting Accuracy: {accuracy_mv:.4f}")

# Simple Averaging
average_preds = (pred1 + pred2 + pred3) / 3
average_classification = np.where(average_preds >= 0.5, 1, 0)
accuracy_avg = accuracy_score(y_test, average_classification)
auc_score_avg = roc_auc_score(y_test, average_preds)

print(f"Simple Averaging Accuracy: {accuracy_avg:.4f}")
print(f"Simple Averaging AUC Score: {auc_score_avg:.4f}")


Majority Voting Accuracy: 0.9312
Simple Averaging Accuracy: 0.9356
Simple Averaging AUC Score: 0.9854
